In [7]:
#conda activate AP1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from scipy.stats import skew, kurtosis

import pyfolio as pf
import empyrical as emp
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from torch.autograd import Variable
from sklearn.model_selection import train_test_split

In [8]:
df = pd.read_excel('data_nn.xlsx')
#df.to_pickle("data_nn.xlsx")

In [9]:
# Set the first column as the date index
df.set_index(df.columns[0], inplace=True)

# Convert the index to string and then to DatetimeIndex format
df.index = pd.to_datetime(df.index.astype(str))

# Filter the data for the last ten years
df_last_10_years = df.loc[df.index > "2020-01-02"]

# Apply rolling sum with a window of 252 and require at least 126 non-NaN values
df_rolling_sum = df_last_10_years.rolling(window=252, min_periods=int(252//2)).sum()

# Forward-fill NaN values, but limit this to a maximum of 5 consecutive fills
df_filled = df_last_10_years.ffill(limit=5)

# Drop any remaining NaN values that still exist after the forward-fill operation
df_cleaned = df_filled.dropna()

#return back original name to not interruppt code.
df_last_10_years = df_cleaned




In [10]:
def refactored_advanced_features(df_returns):
    """
    Refactored computation of advanced financial features to reduce DataFrame fragmentation.
    """
    skew = {}
    kurtosis = {}
    max_drawdown = {}
    volatility = {}
    vaR = {}
    momentum = {}
    avg_return = {}
    rsi = {}

        
        # 1. Skewness
    print("Skewness")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        skew[window] = df_returns.rolling(window).skew()

        # 2. Kurtosis
    print("Kurtosis")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        kurtosis[window]=df_returns.rolling(window).kurt()
    
    # 3. Maximum drawdown
    print("Maximum drawdown")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        max_drawdown[window] = df_returns.rolling(window).apply(emp.max_drawdown, raw=True)
    
    # 4. Volatility
    print("Volatility")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        volatility[window] = df_returns.rolling(window).std()*(252**0.5)
    
    # 5. Value at Risk
    print("Value at Risk")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        vaR[window] = df_returns.rolling(window).apply(emp.value_at_risk, raw=True)
    
    # 6. Momentum
    print("Momentum")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        momentum[window] = df_returns.rolling(window).sum() # ?

    print("Average Return")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        avg_return[window] = df_returns.rolling(window).mean()
    
    return skew, kurtosis, max_drawdown, volatility, vaR, momentum, avg_return

# This function reduces DataFrame fragmentation by constructing all columns and concatenating them at once.

# Läs tommys mex hur de gjorde reversal, sen implementera det. Fixa windows size till vad de hade i rapporten.
# skew[20].head() 

In [11]:
# Call the function and capture the output
skew, kurtosis, max_drawdown, volatility, vaR, momentum, avg_return = refactored_advanced_features(df_last_10_years)
print(skew)


Skewness
Kurtosis
Maximum drawdown
Volatility
Value at Risk
Momentum
Average Return
{20:             Equities_0  Equities_1  Equities_2  Equities_3  Equities_4  \
Column1                                                                  
2020-01-06         NaN         NaN         NaN         NaN         NaN   
2020-01-07         NaN         NaN         NaN         NaN         NaN   
2020-01-08         NaN         NaN         NaN         NaN         NaN   
2020-01-09         NaN         NaN         NaN         NaN         NaN   
2020-01-10         NaN         NaN         NaN         NaN         NaN   
...                ...         ...         ...         ...         ...   
2023-02-20   -0.240833   -0.198584    0.748803   -0.034554    0.183471   
2023-02-21   -0.086253   -0.070400    0.773535    0.011176    0.312154   
2023-02-22   -0.128589   -0.062873    0.757504   -0.109515    0.415347   
2023-02-23   -0.023004    0.106261    0.830820   -0.006954    0.265489   
2023-02-24    0.050596 

In [12]:
# Now, create a DataFrame for each feature by concatenating along the columns
features_df_list = []  # A list to store each feature DataFrame
windows = [20, 40, 60, 100, 180, 240, 360, 480]  # Assuming all features use the same windows

# Iterate through each feature dictionary and create a DataFrame
for feature_name, feature_dict in [('skew', skew), ('kurtosis', kurtosis), ('max_drawdown', max_drawdown), 
                                   ('volatility', volatility), ('vaR', vaR), ('momentum', momentum), ('avg_return', avg_return)]:
    # Only keep the windows that are present for each feature
    relevant_windows = windows if feature_name != 'kurtosis' else windows[:-1]
    feature_df = pd.concat({f'{feature_name}_{window}': feature_dict[window] for window in relevant_windows}, axis=1)
    features_df_list.append(feature_df)

# Concatenate all feature DataFrames into a single DataFrame
features = pd.concat(features_df_list, axis=1)

# Your features_df now contains all the features, with each feature's column named as 'featurename_window'
#print(features_df.tail())  # To print the first 5 rows of the DataFrame
print(features.shape)
print(features)
model.summary()

(822, 3080)
              skew_20                                                         \
           Equities_0 Equities_1 Equities_2 Equities_3 Equities_4 Equities_5   
Column1                                                                        
2020-01-06        NaN        NaN        NaN        NaN        NaN        NaN   
2020-01-07        NaN        NaN        NaN        NaN        NaN        NaN   
2020-01-08        NaN        NaN        NaN        NaN        NaN        NaN   
2020-01-09        NaN        NaN        NaN        NaN        NaN        NaN   
2020-01-10        NaN        NaN        NaN        NaN        NaN        NaN   
...               ...        ...        ...        ...        ...        ...   
2023-02-20  -0.240833  -0.198584   0.748803  -0.034554   0.183471  -0.216372   
2023-02-21  -0.086253  -0.070400   0.773535   0.011176   0.312154  -0.041571   
2023-02-22  -0.128589  -0.062873   0.757504  -0.109515   0.415347   0.052482   
2023-02-23  -0.023004   0.10

NameError: name 'model' is not defined

In [ ]:
# Access the skew dictionary for the 20-day window and print the first few values
print("Last values of skew_20:")
print(skew[20].tail())

Last values of skew_20:
            Equities_0  Equities_1  Equities_2  Equities_3  Equities_4  \
Column1                                                                  
2023-02-20   -0.240833   -0.198584    0.748803   -0.034554    0.183471   
2023-02-21   -0.086253   -0.070400    0.773535    0.011176    0.312154   
2023-02-22   -0.128589   -0.062873    0.757504   -0.109515    0.415347   
2023-02-23   -0.023004    0.106261    0.830820   -0.006954    0.265489   
2023-02-24    0.050596    0.138392    0.864532    0.148482    0.428164   

            Equities_5  Equities_6  Equities_7  Equities_8  Equities_9  ...  \
Column1                                                                 ...   
2023-02-20   -0.216372    0.256436   -0.157259    1.524128    0.163826  ...   
2023-02-21   -0.041571    0.221016   -0.162284    1.521845   -0.296421  ...   
2023-02-22    0.052482    0.191846   -0.470390    1.486008   -0.301710  ...   
2023-02-23    0.092309    0.325509   -0.476984    1.454702   -

In [ ]:
def RSI(df_returns, window):
    """
    Computes the Relative Strength Index (RSI) for a given window.
    """
    df = df_returns.copy()
    df[df >= 0] = 1
    df[df < 0] = 0
    df = df.rolling(window).mean()*100
    return df

RSI skip for now

In [ ]:
# Initialize an empty dictionary to store the last RSI value for each window
rsi_values = {}

# Calculate RSI for each window and store the last value
for window in [20, 40, 60, 100, 180, 240, 360, 480]:
    rsi_df = RSI(df_last_10_years, window)  # df_returns is your DataFrame with returns data
    last_rsi_value = rsi_df.iloc[-1]  # Get the last row of the RSI DataFrame
    rsi_values[window] = last_rsi_value  # Store it in the dictionary with the window as the key

# Print the last RSI value for a 20-day window
print("Last RSI value for 20-day window:")
print(rsi_values[20])




NN

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchinfo
from torchinfo import summary
from sklearn.preprocessing import StandardScaler
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

class PortfolioOptimizationNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(PortfolioOptimizationNN, self).__init__()  # Corrected super() call
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.hidden_layer = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = F.leaky_relu(self.input_layer(x))
        x = F.softmax(self.hidden_layer(x))
        return x

# Using actual columns from the dataset to populate 'dataframes_to_concat'
# As per the uploaded dataset, 'Equities_0', 'Equities_1', and 'Equities_2' are used
# Assume that calculated_features_df is a DataFrame containing all the calculated features
#calculated_features:  skew, kurtosis, max_drawdown, volatility, vaR, momentum, avg_return
#calculated_features = feature_df

calculated_features = features 
calculated_features_df = pd.DataFrame(calculated_features)
dataframes_to_concat = [calculated_features_df]

# Concatenate feature data frames 
features_df = pd.concat(dataframes_to_concat, axis=1)
features_df.fillna(features_df.mean(), inplace=True)

returns = df_last_10_years.drop(columns=df_last_10_years.columns[0])


# Split the data into training and testing sets (considering all columns for y)
X_train, X_test, y_train, y_test = train_test_split(features_df, returns, test_size=0.2, random_state=42)

# Scaling the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Converting to PyTorch tensor
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train.values)  # Multiple columns are included
X_test_tensor = torch.FloatTensor(X_test_scaled)

#print(y_train_tensor.dim)
# Initialize the neural network model
input_dim = X_train_tensor.shape[1] # Specify the number of input features
hidden_dim = 32  # Specify the number of neurons in the hidden layer
#output_dim = 55  # Specify the number of assets or allocation decisions
output_dim = y_train.shape[1]  # Number of columns to predict

model = PortfolioOptimizationNN(input_dim, hidden_dim, output_dim)

""" # Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

print(outputs)
w = model.hidden_layer.weight.data.numpy()
print(w.shape) """

# Function to calculate Sharpe ratio
def sharpe_ratio(weights, returns):
    portfolio_return = torch.sum(weights * returns)
    portfolio_std = torch.std(portfolio_return)
    portfolio_std = torch.std(portfolio_return)
    if portfolio_std == 0:
        return 0  # Avoid division by zero
    else:
        sharpe = torch.mean(portfolio_return) / portfolio_std
        return sharpe
        
""" def sharpe_ratio(weights, returns):
    portfolio_return = torch.sum(weights * returns, dim=1)
    portfolio_std = torch.std(returns, dim=1, unbiased=False)
    
    nan_mask = torch.isnan(portfolio_std) | torch.isinf(portfolio_std) | (portfolio_std == 0)
    portfolio_std[nan_mask] = 1.0  # Set portfolio_std to a non-zero value to avoid division by zero
    
    sharpe = torch.mean(portfolio_return) / torch.mean(portfolio_std)
    return sharpe """

# Loss and optimizer using Sharpe ratio
def sharpe_loss(weights, returns):
    return -sharpe_ratio(weights, returns)

optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop with Sharpe ratio as objective function
epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    sharpe_loss_value = sharpe_loss(outputs, y_train_tensor)  
    sharpe_loss_value.backward()
    optimizer.step()

print(outputs)


C:\Users\sebbe\AppData\Local\Temp\ipykernel_18288\2813019425.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.hidden_layer(x))


tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]], grad_fn=<SoftmaxBackward0>)


In [ ]:
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
# Define the neural network model to accommodate multiple output dimensions
class MultivariateNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MultivariateNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_dim)  # Output dimension should match the number of columns
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

    
# Using actual columns from the dataset to populate 'dataframes_to_concat'
# As per the uploaded dataset, 'Equities_0', 'Equities_1', and 'Equities_2' are used
# Assume that calculated_features_df is a DataFrame containing all the calculated features
#calculated_features: skew, kurtosis, max_drawdown, volatility, vaR, momentum, avg_return
calculated_features = feature_df
calculated_features_df = pd.DataFrame(calculated_features)
dataframes_to_concat = [calculated_features_df]

# Concatenate feature data frames 
features_df = pd.concat(dataframes_to_concat, axis=1)
features_df.fillna(features_df.mean(), inplace=True)


# Split the data into training and testing sets (considering all columns for y)
X_train, X_test, y_train, y_test = train_test_split(features_df, features_df, test_size=0.2, random_state=42)

# Scaling the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(f"Debug: NaN values in training data: {np.isnan(X_train_scaled).sum()}")

# Converting to PyTorch tensor
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.FloatTensor(y_train.values)  # Multiple columns are included
X_test_tensor = torch.FloatTensor(X_test_scaled)

# Initialize the neural network model
output_dim = y_train.shape[1]  # Number of columns to predict
model = MultivariateNN(X_train.shape[1], output_dim)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

initial_weights = model.fc1.weight.data.numpy()
print(f"Debug: Initial weights are {initial_weights}")

# Added check for empty list before DataFrame concatenation

# Check if dataframes_to_concat is empty before attempting concatenation
if len(dataframes_to_concat) == 0:
    print("Warning: No DataFrames to concatenate.")
else:
    features_df = pd.concat(dataframes_to_concat, axis=1)

# Placeholder for Backward Neural Network
class BackwardNN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BackwardNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)
        
# # Initialize the backward neural network model
# backward_model = BackwardNN(X_train.shape[1], output_dim)

# # Training loop for backward neural network
# for epoch in range(epochs):
#     backward_model.train()
#     optimizer.zero_grad()
#     backward_outputs = backward_model(X_train_tensor)
#     backward_loss = criterion(backward_outputs, y_train_tensor)
#     backward_loss.backward()
#     optimizer.step()

# After training
first_layer_weights = model.fc1.weight.data.numpy()
feature_importance = np.abs(first_layer_weights).sum(axis=0)
first_layer_weights = model.fc1.weight.data.numpy()
if first_layer_weights.size == 0:
    print("Debug: The first layer weights are empty.")
else:
    print(f"Debug: The first layer weights have a shape of {first_layer_weights.shape}.")

# Debug Statement  Calculate and display feature importance
feature_importance = np.abs(first_layer_weights).sum(axis=0)
if feature_importance.size == 0:
    print("Debug: Feature importance calculation resulted in an empty array.")
else:
    print(f"Debug: Feature importance is {feature_importance}.")

first_layer_weights.sum()
print(first_layer_weights)

Debug: NaN values in training data: 0
Debug: Initial weights are [[-0.0385122  -0.03669768 -0.09881764 ... -0.00255532  0.01018891
  -0.02646517]
 [ 0.06243932 -0.013294   -0.0128542  ...  0.0762691   0.00617743
  -0.01176683]
 [ 0.02850508  0.10932658  0.0267596  ...  0.05283268  0.00696692
  -0.01801311]
 ...
 [ 0.09273787  0.10088807  0.10259645 ... -0.00867196  0.04689029
   0.05108363]
 [-0.01626513 -0.06953072 -0.06735848 ... -0.01391495 -0.00804681
  -0.04232019]
 [-0.11523475  0.01555605 -0.04022306 ... -0.00133396 -0.06254707
  -0.02541336]]
Debug: The first layer weights have a shape of (128, 448).
Debug: Feature importance is [6.3402395 5.640949  6.10311   6.500687  6.0304546 6.0001235 6.016855
 6.6045012 6.4490294 6.528371  6.501044  6.4230657 6.0943274 6.6018558
 6.808624  6.575309  6.6935496 5.8791294 6.1934805 6.3421173 6.009083
 5.9624653 6.155373  5.4667253 6.0282645 5.994289  5.855509  6.245815
 6.269051  5.850173  5.78609   6.8872776 5.977404  6.308836  6.121019
 6.1

Risk budgeting benchmark

In [ ]:
import cvxpy as cp
from tqdm import tqdm


# Assuming df_last_10_years contains the daily returns
returns = df_last_10_years.drop(columns=df_last_10_years.columns[0])

# Initialize list to store optimal weights and returns for each t
all_weights = []
all_returns = []

# Number of assets
n_assets = len(returns.columns)

# Risk budget for each asset
b = np.ones(n_assets) / n_assets  # For example, equal risk budgeting

# Arbitrary constant for the constraint
c = 1  # Example value, adjust as needed

# Start from the 21st observation
for t in tqdm(range(54,len(returns) - 1,5)):
    # Data up to time t
    data_t = returns.iloc[:t+1].astype("float16")
    print(data_t.index[-1])
    # Covariance matrix of the returns
    cov_matrix_values = data_t.cov().values
    cov_matrix_values = (cov_matrix_values + cov_matrix_values.T)/2

    #cov_matrix_values += np.eye(cov_matrix_values.shape[0],cov_matrix_values.shape[1])

    #if min(eigs) <= 0:
    #    print(t)
    # Portfolio weights as a variable (y)
    y = cp.Variable(shape=n_assets)
    # Objective function: Minimize the square root of the portfolio variance
    objective = cp.Minimize(cp.sqrt(cp.quad_form(y, cp.psd_wrap(cov_matrix_values))))

    # Alternative solutions

    #objective = cp.Minimize(cp.quad_form(y, cp.psd_wrap(cov_matrix_values)))

    #cov_matrix_values = np.array(cov_matrix_values)
    #cov_matrix_values += np.eye(cov_matrix_values.shape[0],cov_matrix_values.shape[1])
    #L = np.linalg.cholesky(cov_matrix_values)
    #objective = cp.Minimize(cp.norm(L@y,2))

    # Constraints:
    # 1. The weights must sum to 1 (full investment)
    # 2. The risk budgeting constraint must be satisfied
    # 3. The weights must be non-negative
    constraints = [
        #cp.sum(y) == 1,        # incompatible with contstraint below. Normalize afterwards
        cp.sum(cp.multiply(b, cp.log(y))) >= c,
        y >= 1e-5 #strict inequalities are not allowed
    ]

    # Formulate the optimization problem
    problem = cp.Problem(objective, constraints)

    # Solve the problem using a suitable solver
    problem.solve(solver=cp.SCS,qcp=True, eps = 1e-5, max_iters  = 100) #kan ange hur många försöja man vill att solve kan göra. Kolla upp det. "Maxiteration". googla på risk budgeting med cvxpy.
    # Store the optimal weights for time t
    optimal_weights = y.value
    #optimal_weights /= np.linalg.norm(optimal_weights,1)
    all_weights.append(optimal_weights)

    #print(optimal_weights)

    #print(type(all_weights))
    #print(type(returns.iloc[t+1].values))
    
    # Calculate and store the portfolio return for time t+1 using weights from time t
    next_return = np.dot(returns.iloc[t+1].values, optimal_weights)
    all_returns.append(next_return)

# Convert lists to arrays for further analysis if needed
#all_weights = np.array(all_weights)
#all_returns = np.array(all_returns)

# Print the first 5 values of all_returns
#print("First 5 values of all_returns:")
#print(all_returns[:5])